In [1]:
# 모든 .ipynb 파일의 시작 부분에 추가
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# training.ipynb

# =========================================================
# 셀 1: 필수 라이브러리 설치 및 초기 설정
# =========================================================
!pip install transformers datasets accelerate pandas scikit-learn torch -q

import os
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# 모델 저장 경로 설정
SAVE_PATH = "/content/drive/MyDrive/Assignment5_Files/my_word_classifier"
MODEL_NAME = "klue/bert-base"
MAX_LENGTH = 64


In [3]:
# =========================================================
# 셀 2: 데이터셋 클래스 정의
# (데이터를 모델이 이해할 수 있는 텐서로 변환하는 역할)
# =========================================================
class WordDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        word = str(self.data.iloc[idx]['word'])
        label = int(self.data.iloc[idx]['label'])

        # 단어를 숫자로 변환 (토큰화)
        inputs = self.tokenizer(
            word,
            padding="max_length",
            truncation=True,
            max_length=MAX_LENGTH,
            return_tensors="pt"
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long)
        }



In [4]:
# =========================================================
# 셀 3: 데이터 로드 및 Train/Val/Test 분할
# (Code 1의 내용)
# =========================================================
# 1. 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/Assignment5_Files/word_labels_final.csv")

# 2. 데이터 청소 (라벨이 비어있는 행 삭제 및 정수 변환)
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)

# 3. 데이터 나누기 (Train 80% : Validation 10% : Test 10%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

# training.ipynb의 데이터 저장 코드 직전에 아래 코드를 추가하세요.

import os # 맨 위에 이미 import 되어 있을 것입니다.

# ... (이전 코드 생략)

# 4. 평가용(Test) 데이터 저장 위치를 Drive로 변경
TEST_CSV_PATH = "/content/drive/MyDrive/Assignment5_Files/test.csv"

# **[⭐️ 여기를 수정/추가하세요 ⭐️]**
# 파일 경로에서 디렉토리만 추출
dir_path = os.path.dirname(TEST_CSV_PATH)
# 디렉토리가 없으면 생성 (exist_ok=True는 이미 폴더가 있어도 오류를 발생시키지 않음)
os.makedirs(dir_path, exist_ok=True)
# **[⭐️ 여기까지 추가 ⭐️]**

test_df.to_csv(TEST_CSV_PATH, index=False)
print(f"👉 Test Set이 '{TEST_CSV_PATH}'에 저장되었습니다.")



👉 Test Set이 '/content/drive/MyDrive/Assignment5_Files/test.csv'에 저장되었습니다.


In [5]:
# =========================================================
# 셀 4: 모델 로드 및 데이터셋 객체 생성
# (Code 2의 내용)
# =========================================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = WordDataset(train_df, tokenizer)
val_dataset = WordDataset(val_df, tokenizer)

# 모델 불러오기 및 GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"현재 사용 중인 장치: {device}")

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

현재 사용 중인 장치: cuda


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
# =========================================================
# 셀 5: 학습 설정 정의 및 학습 시작
# (Code 3의 내용)
# =========================================================
os.environ["WANDB_DISABLED"] = "true" # 외부 로깅 툴 끄기

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    load_best_model_at_end=True,
    learning_rate=2e-5,
    report_to="none",
    weight_decay=0.01,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

print("5. 모델 학습 시작 (약 5 epoch)...")
trainer.train()



5. 모델 학습 시작 (약 5 epoch)...


Epoch,Training Loss,Validation Loss
1,No log,0.491207
2,0.543200,0.422408
3,0.376200,0.602071


TrainOutput(global_step=1116, training_loss=0.44369546828731415, metrics={'train_runtime': 170.8777, 'train_samples_per_second': 52.16, 'train_steps_per_second': 6.531, 'total_flos': 293138604552960.0, 'train_loss': 0.44369546828731415, 'epoch': 3.0})

In [7]:
# =========================================================
# 셀 6: 최종 모델 저장
# =========================================================
SAVE_PATH = "/content/drive/MyDrive/Assignment5_Files/my_word_classifier"
os.makedirs(SAVE_PATH, exist_ok=True)

model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"\n✅ 학습 완료! 모델이 '{SAVE_PATH}' 폴더에 저장되었습니다.")


✅ 학습 완료! 모델이 '/content/drive/MyDrive/Assignment5_Files/my_word_classifier' 폴더에 저장되었습니다.
